# Data Cleaning — Invisible City

**Goal**: Load Toronto chemical release data, clean it, and lock in definitions for "facility" and "risk target."

- Load dataset into pandas
- Clean: missing values, inconsistent chemical names, normalize units
- Define: what is a "facility", what is our "risk target" (facility-level)
- Save clean dataframe to CSV

In [ ]:
import pandas as pd
import numpy as np
import os

# Run with project root as current directory
BASE = os.getcwd()
RAW = os.path.join(BASE, 'data', 'raw')
PROC = os.path.join(BASE, 'data', 'processed')
os.makedirs(PROC, exist_ok=True)

## 1. Load raw data

In [ ]:
facilities = pd.read_csv(os.path.join(RAW, 'facilities_raw.csv'))
releases = pd.read_csv(os.path.join(RAW, 'releases_raw.csv'))
print('Facilities:', facilities.shape)
print('Releases:', releases.shape)
display(facilities.head())
display(releases.head(10))

## 2. Clean releases: missing values, chemical names, units

In [ ]:
# Missing values
print('Missing amount_kg:', releases['amount_kg'].isna().sum())
releases_clean = releases.dropna(subset=['amount_kg']).copy()

# Normalize chemical names (strip, title case)
releases_clean['chemical_name'] = releases_clean['chemical_name'].astype(str).str.strip().str.title()

# Ensure amount is numeric (convert tonnes to kg if your data has it)
releases_clean['amount_kg'] = pd.to_numeric(releases_clean['amount_kg'], errors='coerce')
releases_clean = releases_clean.dropna(subset=['amount_kg'])

print('After cleaning:', releases_clean.shape)

## 3. Definitions

- **Facility**: One row in `facilities` = one reporting site (facility_id).
- **Risk target**: Facility-level. We will assign one risk score per facility.

In [ ]:
# Join facilities with releases for a single clean table (one row per facility with aggregated release info)
facilities_clean = facilities.copy()
agg = releases_clean.groupby('facility_id').agg(
    total_release_kg=('amount_kg', 'sum'),
    release_count=('amount_kg', 'count'),
    n_chemicals=('chemical_name', 'nunique')
).reset_index()
facilities_clean = facilities_clean.merge(agg, on='facility_id', how='left')
facilities_clean['total_release_kg'] = facilities_clean['total_release_kg'].fillna(0)
facilities_clean['release_count'] = facilities_clean['release_count'].fillna(0).astype(int)
facilities_clean['n_chemicals'] = facilities_clean['n_chemicals'].fillna(0).astype(int)

facilities_clean.to_csv(os.path.join(PROC, 'facilities_clean.csv'), index=False)
releases_clean.to_csv(os.path.join(PROC, 'releases_clean.csv'), index=False)
print('Saved facilities_clean.csv, releases_clean.csv')
display(facilities_clean.head())